In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.io as sio
import scipy.interpolate
import matplotlib.pyplot as plt
import os

In [3]:
#Update dataset name as needed


#EXPERIMENT = "sand_sedimented"
#EXPERIMENT = "sand_poured"
#EXPERIMENT = "clay_wider_windows"
EXPERIMENT = 'all'


# RAW_DATA_FOLDER_CLAY = '../'
# RAW_MAT_FOLDER_CLAY = RAW_DATA_FOLDER_CLAY + 'raw_mat_new_clay/'
# RAW_NPY_FOLDER_CLAY = RAW_DATA_FOLDER_CLAY + 'raw_npy_clay/'
# NPY_FOLDER_CLAY = '../processed_input_data_clay/'

RAW_DATA_FOLDER = 'raw_data/'
RAW_MAT_FOLDER = RAW_DATA_FOLDER + 'raw_mat_new_' + EXPERIMENT + '/'
RAW_NPY_FOLDER = RAW_DATA_FOLDER + 'raw_npy_' + EXPERIMENT + '/'
NPY_FOLDER = './processed_input_data_' + EXPERIMENT + '/'

#split data
## attribute all exp_1 to training
## attribute 2 windows (split_ratio) from exp_2 and exp_3 to training
## the rest were randomly used for either eval and train. 
ML_exp = 'split_master'
split_ratio=2

In [4]:
#EXP_LIST contains a list of raw experiment XXX.mat files 
#Format: raw .mat --> raw .npy --> sliced & labeled .npy 

if EXPERIMENT == "clay_wider_windows":
# Experiments list for Clay_wide_windows
   EXP_LIST = ['5_16_19_wide_5w_forCNN', '5_17_19_wide_5w_forCNN', 
               '5_18_19_wide_5w_forCNN', '5_19_19_wide_5w_forCNN',
               '5_20_19_wide_5w_forCNN', '5_22_19_wide_5w_forCNN', 
               '5_23_19_wide_5w_forCNN', '5_24_19_wide_5w_forCNN',
               '5_25_19_wide_5w_forCNN', '6_3_19_wide_5w_forCNN', 
               '6_4_19_wide_5w_forCNN',  '6_7_19_wide_5w_forCNN', 
               '6_13_19_wide_5w_forCNN', '6_18_19_wide_5w_forCNN', 
               '6_19_19_wide_5w_forCNN', '6_24_19_wide_5w_forCNN'] 
    
    
elif EXPERIMENT == "sand_sedimented":
# Experiments list for Sand wider windows
    EXP_LIST = ['E280_64px_w_CNN',
               'E287_64px_w_CNN',
               'E319_64px_w_CNN',
               'E320_64px_w_CNN',
               'E321_64px_w_CNN',
               'E322_64px_w_CNN',
               'E324_64px_w_CNN',
               'E328_64px_w_CNN',
               'E329_64px_w_CNN',
               'E331_64px_w_CNN',
               'E332_64px_w_CNN',
               'E338_64px_w_CNN',
               'E339_64px_w_CNN',
               'E340_64px_w_CNN',
               'E342_64px_w_CNN',
               'E345_64px_w_CNN',
               'E346_64px_w_CNN',
               'E347_64px_w_CNN',
               'E353_64px_w_CNN',
               'E355_64px_w_CNN',
               'E363_64px_w_CNN',
               'E364_64px_w_CNN',
               'E365_64px_w_CNN', 
               'E366_64px_w_CNN',
               'E368_64px_w_CNN',
               'E369_64px_w_CNN',
               'E458_64px_w_CNN',
               'E459_64px_w_CNN',
               'E461_64px_w_CNN',
               'E463_64px_w_CNN',
               'E464_64px_w_CNN',
               'E499_64px_w_CNN']
    
    
else:
    #####use this data split for sand_wide_poured 
#    EXP_LIST= ['E350_64px_w_CNN', 'E351_64px_w_CNN',
#              'E352_64px_w_CNN', 'E439_64px_w_CNN',
#              'E440_64px_w_CNN', 'E441_64px_w_CNN',
#              'E442_64px_w_CNN', 'E444_64px_w_CNN',
#              'E451_64px_w_CNN','E452_64px_w_CNN', 
#               'E455_64px_w_CNN','E456_64px_w_CNN', 
#               'E457_64px_w_CNN','E481_64px_w_CNN', 
#               'E482_64px_w_CNN','E483_64px_w_CNN', 
#               'E484_64px_w_CNN','E494_64px_w_CNN']





### use this data split for all three datasets (sed_sand + poured_sand + clay) 



    EXP_LIST = ['5_16_19_wide_5w_forCNN', '5_17_19_wide_5w_forCNN', 
               '5_18_19_wide_5w_forCNN', '5_19_19_wide_5w_forCNN',
               '5_20_19_wide_5w_forCNN', '5_22_19_wide_5w_forCNN', 
               '5_23_19_wide_5w_forCNN', '5_24_19_wide_5w_forCNN',
               '5_25_19_wide_5w_forCNN', '6_3_19_wide_5w_forCNN', 
               '6_4_19_wide_5w_forCNN',  '6_7_19_wide_5w_forCNN', 
               '6_13_19_wide_5w_forCNN', '6_18_19_wide_5w_forCNN', 
               '6_19_19_wide_5w_forCNN', '6_24_19_wide_5w_forCNN',
               'E280_64px_w_CNN','E287_64px_w_CNN',
               'E319_64px_w_CNN','E320_64px_w_CNN',
               'E321_64px_w_CNN','E322_64px_w_CNN',
               'E324_64px_w_CNN','E328_64px_w_CNN',
               'E329_64px_w_CNN','E331_64px_w_CNN',
               'E332_64px_w_CNN','E338_64px_w_CNN',
               'E339_64px_w_CNN','E340_64px_w_CNN',
               'E342_64px_w_CNN','E345_64px_w_CNN',
               'E346_64px_w_CNN','E347_64px_w_CNN',
               'E353_64px_w_CNN','E355_64px_w_CNN',
               'E363_64px_w_CNN','E364_64px_w_CNN',
               'E365_64px_w_CNN', 'E366_64px_w_CNN',
               'E368_64px_w_CNN','E369_64px_w_CNN',
               'E458_64px_w_CNN','E459_64px_w_CNN',
               'E461_64px_w_CNN','E463_64px_w_CNN',
               'E464_64px_w_CNN','E499_64px_w_CNN', 
               'E350_64px_w_CNN', 'E351_64px_w_CNN',
               'E352_64px_w_CNN', 'E439_64px_w_CNN',
               'E440_64px_w_CNN', 'E441_64px_w_CNN',
               'E442_64px_w_CNN', 'E444_64px_w_CNN',
               'E451_64px_w_CNN','E452_64px_w_CNN', 
               'E455_64px_w_CNN','E456_64px_w_CNN', 
               'E457_64px_w_CNN','E481_64px_w_CNN', 
               'E482_64px_w_CNN','E483_64px_w_CNN', 
               'E484_64px_w_CNN','E494_64px_w_CNN']
                
    


    

In [5]:
# Basic functions to generate.npy files from mat files 


def create_npy(exp):
    matlab_data_path = RAW_MAT_FOLDER + str(exp) + '.mat' 
    npy_data_path = RAW_NPY_FOLDER + str(exp) +'data.npy'
    npy_label_path = RAW_NPY_FOLDER + str(exp) +'label.npy'
    npy_para_path = RAW_NPY_FOLDER + str(exp) +'para.npy'
    
    if not os.path.exists(RAW_NPY_FOLDER):
        os.mkdir(RAW_NPY_FOLDER)
    if os.path.exists(npy_data_path):
        raise Exception("raw .npy file already exist")
    else:  
        data = sio.loadmat(matlab_data_path) ["fault_map"]  
        label = sio.loadmat(matlab_data_path) ["label"]
        
        WW = sio.loadmat(matlab_data_path) ["windowWidth"]
        overlap = sio.loadmat(matlab_data_path) ["overlap"]
        
        slice_param = [0 for a in range(4)]
        slice_param[3] = WW[0,0]
        slice_param[2] = 2* slice_param[3]
        slice_param[1] = slice_param[3]- overlap[0,0]
        slice_param[0] = 0
        
        # report sizes and save data, parameters and label
        print(data.shape)
        print(label.shape)
        print(slice_param)
        np.save(npy_data_path, data)
        np.save(npy_label_path, label)
        np.save(npy_para_path, slice_param)

In [6]:
# Generate and save raw_npy for list of experiment
# Only need to run once! 

for e in EXP_LIST:
    print(e)
    create_npy(e)

5_16_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_17_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_18_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_19_19_wide_5w_forCNN
(128, 208, 191)
(5, 191, 2)
[0, 36, 128, 64]
5_20_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_22_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_23_19_wide_5w_forCNN
(128, 208, 200)
(5, 200, 2)
[0, 36, 128, 64]
5_24_19_wide_5w_forCNN
(128, 208, 189)
(5, 189, 2)
[0, 36, 128, 64]
5_25_19_wide_5w_forCNN
(128, 208, 190)
(5, 190, 2)
[0, 36, 128, 64]
6_3_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)
[0, 36, 128, 64]
6_4_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)
[0, 36, 128, 64]
6_7_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)
[0, 36, 128, 64]
6_13_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)
[0, 36, 128, 64]
6_18_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)
[0, 36, 128, 64]
6_19_19_wide_5w_forCNN
(128, 208, 150)
(5, 150, 2)


In [7]:
#This function creates slices of each fault map
def slicing_hw(RAW_NPY_FOLDER, exp): 
    npy_data_path = RAW_NPY_FOLDER + str(exp) +'data.npy'
    npy_para_path = RAW_NPY_FOLDER + str(exp) +'para.npy'
    
    data = np.load(npy_data_path)
    print(data.shape)
    slice_param = np.load(npy_para_path)
    
    #data has dimension of (x, y, cfuv, t)
    width_tot = data.shape[1]
    t_tot = data.shape[2]
    #print(width_tot,t_tot)
    
    #unpack slice_param into initial_index, strike_lenght, window_H and window_W
    ii = slice_param[0]
    strike = slice_param[1]
    H = slice_param[2]
    W = slice_param[3]
    #print(ii,strike,H,W)
    
    # set number of windows and slice data and label 
    num_win = 1 + (width_tot - W)//strike   
    
    data_hw = []
    w_idx = np.arange(num_win)
    for w_i in w_idx:
        left_win = w_i*strike
        #print(left_win)
        data_hwi = data[:,left_win:left_win+W]
        #data_hw is a list of tuples (number of tuples = num_win)
        #first element of tuple has a shape = (WindowHeight, WindowWidth, numFrames)
        #second element of tuple is the w_idx
        data_hw.append((data_hwi, w_i))

    print("Data_slice_shape =", data_hw[0][0].shape)
    return data_hw, num_win, t_tot


In [8]:
#This function gathers the label information and saves with slices to be read for dividing into 
#training, eval and testing data sets.
def data_slice_KE(RAW_NPY_FOLDER, NPY_FOLDER, exp):
    
    #set file location for 
    npy_exp_folder_path = NPY_FOLDER + 'slice_npy/' + str(exp)
    master_folder_path = NPY_FOLDER + 'file_master/'
    
    #need to grab the KE labels
    npy_label_path = RAW_NPY_FOLDER + str(exp) +'label.npy'
    label = np.load(npy_label_path)
    #print('label', label.shape)
    
    if os.path.exists(npy_exp_folder_path):
        raise Exception("slice_exp.npy already exist")
    else: 
        os.makedirs(npy_exp_folder_path)
        if not os.path.exists(master_folder_path): 
            os.makedirs(master_folder_path)     
    
    #slice the fault_map
    data_hw, num_win, t_tot = slicing_hw(RAW_NPY_FOLDER, exp)
    
    w_idx = np.arange(num_win)
    file_master = []
    
    #looping through all windows and times 
    for w_i in w_idx: 
        for t_i in range(t_tot):
            #unpacking data_hw
            data_f, win = data_hw[w_i]
            #print(win)
        
            #only process slice with non-zero fault traces
            #if np.sum(data_f[:,:,t_i]) != 0:
            #only process slice with KE < 2% because we aren't interested in teeny tiny faults
            #Also the sand experiments have noise so the fault map might seem to have faults but these are just
            #noise and we don't want the noise to factor into the machine learning since the crustal fault maps
            #won't have that type of noise.
            
            
                
            #The first array of the label array are the std and the second array is the KE
            label_KE = label[w_i,t_i,1]
            label_SD = label[w_i,t_i,0]
            print(w_i,t_i,label_KE,label_SD)
            
            if label_KE > 0.02:
                
                    #use next line for sand dataset only
                    name_path = '/' + "{:.4f}".format(label_KE) + '_' + "{:.4f}".format(label_SD) + '_win_' + str('{:02d}'.format(win)) + '_t_' + str('{:03d}'.format(t_i)) + '_' + str(exp) + '_cfuv.npy'
                    
                    #use next line for clay dataset only
                    #name_path = '/' + "{:.4f}".format(label_KE) + '_' + "{:.4f}".format(label_SD) + '_win_' + str(win) + '_t_' + str('{:03d}'.format(t_i)) + '_' + str(exp) + '_cfuv.npy'
                # save sliced and labeled dataset in slice_npy folder
                    np.save(npy_exp_folder_path + name_path, data_f[:,:,t_i])
                   
                #make a warning message if the KE is less than 0.005
            #if label_KE >1.0:
                   # print('uh oh: ', npy_exp_folder_path, label_KE)
                
                # add to file master
                    file_master.append(str(exp) + name_path)          
    
    master_data_path = master_folder_path + str(exp) + ".txt"
    print('non_zero_slice =', len(file_master))
    print (num_win)
    np.savetxt(master_data_path, file_master, fmt="%s")

In [9]:
# Only need to run once! 
for exp in EXP_LIST:
    data_slice_KE(RAW_NPY_FOLDER, NPY_FOLDER, exp)

(128, 208, 200)
Data_slice_shape = (128, 64, 200)
0 0 0.0 0.0
0 1 0.0 0.0
0 2 0.0 0.0
0 3 0.0 0.0
0 4 0.0 0.0
0 5 0.0 0.0
0 6 0.0 0.0
0 7 0.0 0.0
0 8 0.0 0.0
0 9 0.0 0.0
0 10 0.0 0.0
0 11 0.0 0.0
0 12 0.0 0.0
0 13 0.0 0.0
0 14 0.0 0.0
0 15 0.0 0.0
0 16 0.0 0.0
0 17 0.0 0.0
0 18 0.0 0.0
0 19 0.0 0.0
0 20 0.0 0.0
0 21 0.0 0.0
0 22 0.0 0.0
0 23 0.0 0.0
0 24 0.0 0.0
0 25 0.0 0.0
0 26 0.0 0.0
0 27 0.0 0.0
0 28 0.0 0.0
0 29 0.0 0.0
0 30 0.0 0.0
0 31 0.0 0.0
0 32 0.0 0.0
0 33 0.0 0.0
0 34 0.0 0.0
0 35 0.0 0.0
0 36 0.0 0.0
0 37 0.0 0.0
0 38 0.0 0.0
0 39 0.0 0.0
0 40 0.0 0.0
0 41 0.0 0.0
0 42 0.0 0.0
0 43 0.0 0.0
0 44 0.0 0.0
0 45 0.0 0.0
0 46 0.0 0.0
0 47 0.0 0.0
0 48 0.0 0.0
0 49 0.0 0.0
0 50 0.0 0.0
0 51 0.0 0.0
0 52 0.0 0.0
0 53 0.0 0.0
0 54 0.0 0.0
0 55 0.0 0.04829399963826104
0 56 0.0 0.060167192992355255
0 57 0.0 0.07002836603114462
0 58 0.0 0.0744136444173184
0 59 0.027108670248908673 0.08250233434293529
0 60 0.08755769467305506 0.07933490965023386
0 61 0.11443027244530553 0.07980956868

(128, 208, 200)
Data_slice_shape = (128, 64, 200)
0 0 0.0 0.0
0 1 0.0 0.0
0 2 0.0 0.0
0 3 0.0 0.0
0 4 0.0 0.0
0 5 0.0 0.0
0 6 0.0 0.0
0 7 0.0 0.0
0 8 0.0 0.0
0 9 0.0 0.0
0 10 0.0 0.0
0 11 0.0 0.0
0 12 0.0 0.0
0 13 0.0 0.0
0 14 0.0 0.0
0 15 0.0 0.0
0 16 0.0 0.0
0 17 0.0 0.0
0 18 0.0 0.0
0 19 0.0 0.0
0 20 0.0 0.0
0 21 0.0 0.0
0 22 0.0 0.0
0 23 0.0 0.0
0 24 0.0 0.0
0 25 0.0 0.0
0 26 0.0 0.0
0 27 0.0 0.0
0 28 0.0 0.0
0 29 0.0 0.0
0 30 0.0 0.0
0 31 0.0 0.0
0 32 0.0 0.0
0 33 0.0 0.0
0 34 0.0 0.0
0 35 0.0 0.0
0 36 0.0 0.0
0 37 0.0 0.0
0 38 0.0 0.0
0 39 0.0 0.0
0 40 0.0 0.0
0 41 0.0 0.0
0 42 0.0 0.0
0 43 0.0 0.0
0 44 0.0 0.0
0 45 0.0 0.0
0 46 0.0 0.0
0 47 0.0 0.0
0 48 0.0 0.0
0 49 0.0 0.0
0 50 0.0 0.0
0 51 0.0 0.0
0 52 0.0 0.0
0 53 0.0 0.0
0 54 0.0 0.0
0 55 0.0 0.0
0 56 0.0 0.0
0 57 0.0 0.0
0 58 0.0 0.0
0 59 0.0 0.0
0 60 0.0 0.0
0 61 0.0 0.0
0 62 0.0 0.03837220709625004
0 63 0.06819246371619118 0.06351391278029833
0 64 0.08551454832104552 0.0788556694691932
0 65 0.09398254046813828 0.089419252

0 139 0.5431601136044462 0.11212974438142957
0 140 0.5408288595429319 0.1155609407043625
0 141 0.5406083715787937 0.11403818818425271
0 142 0.5350841474366198 0.11429906067996241
0 143 0.5467534919501623 0.11624821181328131
0 144 0.5477536343863378 0.11291576426526945
0 145 0.5487906295396588 0.11515981170749547
0 146 0.5531915784324859 0.11686417721072084
0 147 0.5495258734989703 0.1166277813083473
0 148 0.5810314468266742 0.12647413438058241
0 149 0.5922125595499829 0.1257237742652763
0 150 0.5762116384217261 0.1291731081062718
0 151 0.5804975265407472 0.13073941462851296
0 152 0.5911074638319285 0.13354924267460397
0 153 0.5964776226942555 0.13758662292150264
0 154 0.5890628766209272 0.12976492640908044
0 155 0.5778198012879527 0.12547471034182867
0 156 0.601195973990536 0.12839953126674244
0 157 0.6140295115063595 0.13518114127595623
0 158 0.6019038680852618 0.13141384663064756
0 159 0.6026001904873795 0.1331092918251732
0 160 0.6040256120978197 0.1385107694643355
0 161 0.601746134

1 0 0.0 0.0
1 1 0.0 0.0
1 2 0.0 0.0
1 3 0.0 0.0
1 4 0.0 0.0
1 5 0.0 0.0
1 6 0.0 0.0
1 7 0.0 0.0
1 8 0.0 0.0
1 9 0.0 0.0
1 10 0.0 0.0
1 11 0.0 0.0
1 12 0.0 0.0
1 13 0.0 0.0
1 14 0.0 0.0
1 15 0.0 0.0
1 16 0.0 0.0
1 17 0.0 0.0
1 18 0.0 0.0
1 19 0.0 0.0
1 20 0.0 0.0
1 21 0.0 0.0
1 22 0.0 0.0
1 23 0.0 0.0
1 24 0.0 0.0
1 25 0.0 0.0
1 26 0.0 0.0
1 27 0.0 0.0
1 28 0.0 0.0
1 29 0.0 0.0
1 30 0.0 0.0
1 31 0.0 0.0
1 32 0.0 0.0
1 33 0.0 0.0
1 34 0.0 0.0
1 35 0.0 0.0
1 36 0.0 0.0
1 37 0.0 0.0
1 38 0.0 0.0
1 39 0.0 0.0
1 40 0.0 0.0
1 41 0.0 0.0
1 42 0.0 0.0
1 43 0.0 0.0
1 44 0.0 0.0
1 45 0.0 0.0
1 46 0.0 0.0
1 47 0.0 0.0
1 48 0.0 0.0
1 49 0.0 0.0
1 50 0.0 0.0
1 51 0.0 0.0
1 52 0.0 0.0
1 53 0.0 0.0
1 54 0.0 0.0
1 55 0.0 0.0
1 56 0.0 0.0
1 57 0.0 0.0
1 58 0.0 0.0
1 59 0.0 0.0
1 60 0.0 0.0
1 61 0.0 0.0
1 62 0.0 0.0
1 63 0.0 0.0
1 64 0.0 0.0
1 65 0.0 0.0
1 66 0.0 0.0
1 67 0.0 0.0
1 68 0.0 0.0
1 69 0.0 0.0
1 70 0.0 0.0
1 71 0.0 0.0
1 72 0.0 0.030677103577814183
1 73 0.0 0.04123130122702969
1 74 0.0 0.0499

2 91 0.34301089658225525 0.07225398825325416
2 92 0.3535747560955139 0.07196543251852514
2 93 0.35727852731924015 0.07207274921109418
2 94 0.3685209245568386 0.07117906680528663
2 95 0.3722320362158013 0.07743182976365152
2 96 0.37721636342868403 0.08233100390691038
2 97 0.36459770542552905 0.08889730359943158
2 98 0.40281058843218703 0.09795758926868042
2 99 0.38639127654504446 0.09718573254994717
2 100 0.37879777188912495 0.09647393111919472
2 101 0.3840651349913328 0.09597250189032755
2 102 0.40280207822784575 0.08962517892585645
2 103 0.41357290579420614 0.0983125764268496
2 104 0.4319637124368187 0.09487297100599153
2 105 0.41793621419248056 0.0905370119933832
2 106 0.40633344899301194 0.0917409487415297
2 107 0.41125298899202156 0.08402508444026423
2 108 0.3903079719186716 0.0871440092721118
2 109 0.40418808727946165 0.08276483482357824
2 110 0.39368198518684644 0.08898652234991916
2 111 0.39000257533117727 0.08648651451362326
2 112 0.38612777648657376 0.09115535686665663
2 113 0

...

0 48 0.522720217704773 0.2236717939376831
0 49 0.5709173679351807 0.21776799857616425
0 50 0.559333086013794 0.22529961168766022
0 51 0.5719399452209473 0.21881794929504395
0 52 0.6101956367492676 0.22075173258781433
0 53 0.6252396106719971 0.23426246643066406
0 54 0.560132622718811 0.22522161900997162
0 55 0.6132500767707825 0.23306915163993835
0 56 0.5813882350921631 0.22832676768302917
0 57 0.5844647884368896 0.2329101711511612
0 58 0.5787672996520996 0.2321372926235199
0 59 0.5895010232925415 0.23199862241744995
0 60 0.5740034580230713 0.23621410131454468
0 61 0.5626447200775146 0.31184959411621094
0 62 0.524031400680542 0.22865904867649078
0 63 0.5443335771560669 0.22496038675308228
0 64 0.5727078914642334 0.2401461899280548
0 65 0.5618705749511719 0.24862655997276306
0 66 0.5367435216903687 0.24712705612182617
0 67 0.5428153872489929 0.23488160967826843
0 68 0.515906572341919 0.24643416702747345
0 69 0.552967369556427 0.2394602745771408
0 70 0.5688936710357666 0.24446050822734833

5 28 0.6001630425453186 0.24656441807746887
5 29 0.6108547449111938 0.24919353425502777
5 30 0.6160974502563477 0.23870277404785156
5 31 0.6456050276756287 0.20826157927513123
5 32 0.6607790589332581 0.19083458185195923
5 33 0.6582092046737671 0.1541820764541626
5 34 0.6726419925689697 0.14399290084838867
5 35 0.6887964010238647 0.13556596636772156
5 36 0.6810905933380127 0.13203442096710205
5 37 0.6910489797592163 0.12602351605892181
5 38 0.694844126701355 0.12226156145334244
5 39 0.6951189041137695 0.12206336855888367
5 40 0.7270157337188721 0.11767342686653137
5 41 0.708501935005188 0.13157297670841217
5 42 0.7455464601516724 0.13415175676345825
5 43 0.7497279644012451 0.1547798365354538
5 44 0.7422152757644653 0.18419508635997772
5 45 0.7348171472549438 0.16420692205429077
5 46 0.7443806529045105 0.1861523687839508
5 47 0.7232725024223328 0.16618233919143677
5 48 0.7332801818847656 0.13283851742744446
5 49 0.7441005110740662 0.1519712656736374
5 50 0.7321966290473938 0.152485281229

1 24 0.4492952227592468 0.2094154953956604
1 25 0.4816136062145233 0.2152814120054245
1 26 0.44669419527053833 0.20970748364925385
1 27 0.47792723774909973 0.24758444726467133
1 28 0.5124053955078125 0.1876235455274582
1 29 0.5034579038619995 0.20676176249980927
1 30 0.4721478521823883 0.210530087351799
1 31 0.4963539242744446 0.20700842142105103
1 32 0.5119545459747314 0.1954220086336136
1 33 0.5131409168243408 0.16924868524074554
1 34 0.521348237991333 0.15346397459506989
1 35 0.5381646156311035 0.14747273921966553
1 36 0.5515565872192383 0.14954088628292084
1 37 0.5614794492721558 0.13963651657104492
1 38 0.5566531419754028 0.14626003801822662
1 39 0.5547424554824829 0.146572545170784
1 40 0.5701969861984253 0.15288278460502625
1 41 0.6146318912506104 0.1546400487422943
1 42 0.6096854209899902 0.16242164373397827
1 43 0.6037722826004028 0.24023917317390442
1 44 0.580543041229248 0.2407113015651703
1 45 0.609387993812561 0.24507853388786316
1 46 0.5905274152755737 0.2471020668745041


6 24 0.5864125490188599 0.15937556326389313
6 25 0.54969322681427 0.14230594038963318
6 26 0.5554898381233215 0.1408429741859436
6 27 0.5962320566177368 0.1661180555820465
6 28 0.6286997199058533 0.18063883483409882
6 29 0.6500343084335327 0.20161060988903046
6 30 0.6255419850349426 0.19780927896499634
6 31 0.6217010021209717 0.19535835087299347
6 32 0.6256245374679565 0.1942373514175415
6 33 0.6126525402069092 0.19265353679656982
6 34 0.6515089273452759 0.19052894413471222
6 35 0.6508983373641968 0.2023790031671524
6 36 0.6872351765632629 0.15786314010620117
6 37 0.7033682465553284 0.12947498261928558
6 38 0.7259326577186584 0.12568223476409912
6 39 0.7562516927719116 0.0918717309832573
6 40 0.7693657875061035 0.10259275138378143
6 41 0.7574729919433594 0.10043377429246902
6 42 0.7809314131736755 0.09649280458688736
6 43 0.7684135437011719 0.08812310546636581
6 44 0.7834148406982422 0.07428331673145294
6 45 0.7885016202926636 0.07821766287088394
6 46 0.7874671220779419 0.0634535178542

1 69 0.7388491034507751 0.09778405725955963
1 70 0.7557132244110107 0.10544285923242569
1 71 0.7346335053443909 0.11031130701303482
1 72 0.7589026689529419 0.10705738514661789
1 73 0.7440999746322632 0.11542209982872009
2 0 0.0 0.0
2 1 0.0 0.0
2 2 0.0 0.0
2 3 0.0 0.0
2 4 0.0 0.0
2 5 0.0 0.0
2 6 0.0 0.0
2 7 0.10832875967025757 0.0870325118303299
2 8 0.1454019397497177 0.09966696798801422
2 9 0.1889457106590271 0.09569184482097626
2 10 0.22105348110198975 0.10734923928976059
2 11 0.27002277970314026 0.13898110389709473
2 12 0.2925076186656952 0.15681475400924683
2 13 0.3766937255859375 0.16281183063983917
2 14 0.418123722076416 0.14468339085578918
2 15 0.4469318091869354 0.14220722019672394
2 16 0.47185826301574707 0.13278499245643616
2 17 0.4845266342163086 0.09888099133968353
2 18 0.5027973651885986 0.10146147012710571
2 19 0.5646700859069824 0.0989827811717987
2 20 0.6155098080635071 0.12140069156885147
2 21 0.6257635354995728 0.13169750571250916
2 22 0.6280496120452881 0.170035570859

In [10]:
################## USE SPLIT FOR SAND ONLY #################################

def create_dataset(DATA_FOLDER,ML_exp, G1, G2, split_ratio):
    train_combine = []
    eval_combine = []
    test_combine = []
    eval_exp_win_stat = []
    test_exp_win_stat = []
    train_exp_win_stat = []
    for exp in G1:        
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        a = np.loadtxt(fmt, dtype=str) 
        train_combine = np.concatenate([train_combine, a])
        # housekeeping for the exp/slice included in training
        num_win1 = int(a[-1].split('win_')[-1][0:2])+1 
        for i in range (num_win1):
            train_slice = [exp, i]
            train_exp_win_stat.append(train_slice)
        #check how many time slice per window 
#        win_len = len(a)/num_win1
#        print("num_win",num_win1)
#        print("len_a",len(a))
        #check randomized which slice would be included in 
#        shf_idx = np.arange(num_win1)
#        print(shf_idx)
#        np.random.shuffle(shf_idx)
#        print(shf_idx)


#        split_eval=num_win1//3            
        
        #distribute one random window-slice to eval
#        for idx in range(split_eval-1):
#            eval_ini = int(shf_idx[idx]*win_len)
#            eval_end = int((shf_idx[idx]+1)*win_len)
#            eval_combine = np.concatenate([eval_combine,  a[eval_ini:eval_end]])
            # housekeeping for the exp/slice included in evaluating
#            eval_slice = [exp,shf_idx[idx]] 
#            eval_exp_win_stat.append(eval_slice)
                
        #distribute one random window-slice to test
#        test_ini = int((shf_idx[split_eval-1])*win_len)
#        test_end = int((shf_idx[split_eval-1]+1)*win_len)
#        test_combine = np.concatenate([test_combine,  a[test_ini:test_end]])
        # houseckeeping for the exp/slice included in testting
#        test_slice = [exp,shf_idx[split_eval-1]] 
#        test_exp_win_stat.append(test_slice)
        
        #distribute the rest of random window-slices to train
#        for idx in range(split_eval,num_win1):
#            train_ini = int(shf_idx[idx]*win_len)
#            train_end = int((shf_idx[idx]+1)*win_len)
#            train_combine = np.concatenate([train_combine, a[train_ini:train_end]])
           ## houseckeeping for the exp/slice included in training
#            train_slice = [exp,shf_idx[idx]] 
#            train_exp_win_stat.append(train_slice) 
            
            
    for exp in G2:
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        b = np.loadtxt(fmt, dtype=str)
        #check last element for its window number + 1 to get #of window
        num_win2 = int(b[-1].split('win_')[-1][0:2])+1
        #check how many time slice per window 
        win_len = len(b)/num_win2
        print("num_win",num_win2)
        print("len_b",len(b))
        #check randomized which slice would be included in 
        shf_idx = np.arange(num_win2)
        print(shf_idx)
        np.random.shuffle(shf_idx)
        print(shf_idx)


        split_eval=num_win2//3
        #split_eval=2
              
        
        #distribute one random window-slice to eval
        for idx in range(split_eval):
            eval_ini = int(shf_idx[idx]*win_len)
            eval_end = int((shf_idx[idx]+1)*win_len)
            eval_combine = np.concatenate([eval_combine,  b[eval_ini:eval_end]])
            # housekeeping for the exp/slice included in evaluating
            eval_slice = [exp,shf_idx[idx]] 
            eval_exp_win_stat.append(eval_slice)
                
        #distribute one random window-slice to test
        test_ini = int((shf_idx[split_eval])*win_len)
        test_end = int((shf_idx[split_eval]+1)*win_len)
        test_combine = np.concatenate([test_combine,  b[test_ini:test_end]])
        # houseckeeping for the exp/slice included in testting
        test_slice = [exp,shf_idx[split_eval]] 
        test_exp_win_stat.append(test_slice)
        
        #distribute the rest of random window-slices to train
        for idx in range(split_eval+1,num_win2):
            train_ini = int(shf_idx[idx]*win_len)
            train_end = int((shf_idx[idx]+1)*win_len)
            train_combine = np.concatenate([train_combine, b[train_ini:train_end]])
            ## houseckeeping for the exp/slice included in training
            train_slice = [exp,shf_idx[idx]] 
            train_exp_win_stat.append(train_slice) 
            

     
    tot_dataset = len(train_combine) + len(eval_combine) + len(test_combine)
    train_ratio = format(len(train_combine)/tot_dataset, '.2f')
    eval_ratio = format(len(eval_combine)/tot_dataset, '.2f')
    test_ratio = format(len(test_combine)/tot_dataset, '.2f')
    
    stat_dict = {'train_ew': train_exp_win_stat,
                'eval_ew': eval_exp_win_stat,
                'test_ew': test_exp_win_stat,
                'tot_dataset':tot_dataset,
                'train_ratio':train_ratio,
                'eval_ratio':eval_ratio,
                'test_ratio':test_ratio}
    print(stat_dict)
    ML_EXP_PATH = DATA_FOLDER + str(ML_exp)
    if not os.path.exists(ML_EXP_PATH): 
        os.makedirs(ML_EXP_PATH)
    
    with open(ML_EXP_PATH +'/data_stat.txt', 'w') as f:
        print(stat_dict, file=f)
        
    train_data_path = ML_EXP_PATH +  '/train_master.txt'
    np.savetxt(train_data_path, train_combine, fmt="%s")
    
    eval_data_path = ML_EXP_PATH +  '/eval_master.txt'
    np.savetxt(eval_data_path, eval_combine, fmt="%s")
    
    test_data_path = ML_EXP_PATH +  '/test_master.txt'
    np.savetxt(test_data_path, test_combine, fmt="%s")
    return stat_dict

In [45]:
######################### USE THIS SPLIT FOR CLAY ONLY #######################

split_ratio=2
def create_dataset(DATA_FOLDER,ML_exp, G1, G2, split_ratio):
    train_combine = []
    eval_combine = []
    test_combine = []
    eval_exp_win_stat = []
    test_exp_win_stat = []
    train_exp_win_stat = []
    for exp in G1:        
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        a = np.loadtxt(fmt, dtype=str)       
        # for each experiments in G1, we 100% attribute them to training dataset. 
        train_combine = np.concatenate([train_combine, a])
        # housekeeping for the exp/slice included in training
        num_win = int(a[-1].split('win_')[-1][0:1])+1
        for i in range (num_win):
            train_slice = [exp, i]
            train_exp_win_stat.append(train_slice)
            
    for exp in G2:
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        b = np.loadtxt(fmt, dtype=str)
        #check last element for its window number + 1 to get #of window
        num_win = int(b[-1].split('win_')[-1][0:1])+1
        #check how many time slice per window 
        win_len = len(b)/num_win
        
        #check randomized which slice would be included in 
        shf_idx = np.arange(num_win)
        print(shf_idx)
        np.random.shuffle(shf_idx)
        print(shf_idx)
        
        split_eval = split_ratio
        
        #distribute two random window-slice to eval
        for idx in range(split_eval):
            eval_ini = int(shf_idx[idx]*win_len)
            eval_end = int((shf_idx[idx]+1)*win_len)
            eval_combine = np.concatenate([eval_combine,  b[eval_ini:eval_end]])
            # housekeeping for the exp/slice included in evaluating
            eval_slice = [exp,shf_idx[idx]] 
            eval_exp_win_stat.append(eval_slice)
                
        #distribute one random window-slice to test
        test_ini = int(shf_idx[split_eval]*win_len)
        test_end = int((shf_idx[split_eval]+1)*win_len)
        test_combine = np.concatenate([test_combine,  b[test_ini:test_end]])
        # houseckeeping for the exp/slice included in testting
        test_slice = [exp,shf_idx[split_eval]] 
        test_exp_win_stat.append(test_slice)
        
        #distribute the rest of random window-slices to train
        for idx in range(split_eval+1,num_win):
            train_ini = int(shf_idx[idx]*win_len)
            train_end = int((shf_idx[idx]+1)*win_len)
            train_combine = np.concatenate([train_combine, b[train_ini:train_end]])
            # houseckeeping for the exp/slice included in training
            train_slice = [exp,shf_idx[idx]] 
            train_exp_win_stat.append(train_slice)
            
    tot_dataset = len(train_combine) + len(eval_combine) + len(test_combine)
    train_ratio = format(len(train_combine)/tot_dataset, '.2f')
    eval_ratio = format(len(eval_combine)/tot_dataset, '.2f')
    test_ratio = format(len(test_combine)/tot_dataset, '.2f')
    
    stat_dict = {'train_ew': train_exp_win_stat,
                'eval_ew': eval_exp_win_stat,
                'test_ew': test_exp_win_stat,
                'tot_dataset':tot_dataset,
                'train_ratio':train_ratio,
                'eval_ratio':eval_ratio,
                'test_ratio':test_ratio}
    print(stat_dict)
    ML_EXP_PATH = DATA_FOLDER + str(ML_exp)
    if not os.path.exists(ML_EXP_PATH): 
        os.makedirs(ML_EXP_PATH)
    
    with open(ML_EXP_PATH +'/data_stat.txt', 'w') as f:
        print(stat_dict, file=f)
        
    train_data_path = ML_EXP_PATH +  '/train_master.txt'
    np.savetxt(train_data_path, train_combine, fmt="%s")
    
    eval_data_path = ML_EXP_PATH +  '/eval_master.txt'
    np.savetxt(eval_data_path, eval_combine, fmt="%s")
    
    test_data_path = ML_EXP_PATH +  '/test_master.txt'
    np.savetxt(test_data_path, test_combine, fmt="%s")
    return stat_dict

In [12]:

    
if EXPERIMENT == "clay_wider_windows":
    G1 = ['5_17_19_wide_5w_forCNN', '5_16_19_wide_5w_forCNN', '5_19_19_wide_5w_forCNN',
         '6_7_19_wide_5w_forCNN', '6_19_19_wide_5w_forCNN', '6_4_19_wide_5w_forCNN',
         '5_24_19_wide_5w_forCNN','5_18_19_wide_5w_forCNN', '5_20_19_wide_5w_forCNN']
         
    
    G2 = ['5_22_19_wide_5w_forCNN', '6_18_19_wide_5w_forCNN',
          '5_23_19_wide_5w_forCNN', '5_25_19_wide_5w_forCNN', 
          '6_24_19_wide_5w_forCNN', '6_13_19_wide_5w_forCNN', '6_3_19_wide_5w_forCNN']
    create_dataset(NPY_FOLDER, ML_exp, G1, G2,split_ratio)    
    

elif EXPERIMENT=="sand_sedimented":
    G1 = ['E280_64px_w_CNN',
          'E287_64px_w_CNN',
          'E329_64px_w_CNN',
          'E331_64px_w_CNN',
          'E459_64px_w_CNN',
          'E464_64px_w_CNN',
          'E499_64px_w_CNN',
          'E340_64px_w_CNN',
          'E342_64px_w_CNN',
          'E364_64px_w_CNN',
          'E365_64px_w_CNN']
        
    G2 = ['E319_64px_w_CNN',
          'E320_64px_w_CNN',
          'E321_64px_w_CNN',
          'E322_64px_w_CNN',
          'E324_64px_w_CNN',
          'E328_64px_w_CNN',
          'E332_64px_w_CNN',
          'E345_64px_w_CNN',
          'E346_64px_w_CNN',
          'E347_64px_w_CNN',
          'E353_64px_w_CNN',
          'E355_64px_w_CNN',
          'E458_64px_w_CNN',
          'E461_64px_w_CNN',
          'E463_64px_w_CNN',
          'E338_64px_w_CNN',
          'E339_64px_w_CNN',
          'E363_64px_w_CNN',
          'E366_64px_w_CNN',
          'E368_64px_w_CNN',
          'E369_64px_w_CNN']

    create_dataset(NPY_FOLDER, ML_exp, G1, G2,split_ratio)
    
    
else:
    
    ###data split for poured sand
#    G1=['E351_64px_w_CNN', 'E352_64px_w_CNN',
#        'E440_64px_w_CNN', 'E451_64px_w_CNN', 
#        'E455_64px_w_CNN']
#    
#    G2=['E350_64px_w_CNN','E439_64px_w_CNN', 
#        'E441_64px_w_CNN', 'E442_64px_w_CNN', 
#        'E444_64px_w_CNN', 'E452_64px_w_CNN',
#        'E456_64px_w_CNN', 'E457_64px_w_CNN',
#        'E481_64px_w_CNN', 'E482_64px_w_CNN', 
#        'E483_64px_w_CNN', 'E484_64px_w_CNN',
#        'E494_64px_w_CNN']
#      
#    create_dataset(NPY_FOLDER, ML_exp, G1, G2,split_ratio)



    ###data split for all three datasets
    
    G1 = ['5_17_19_wide_5w_forCNN', '5_16_19_wide_5w_forCNN', '5_19_19_wide_5w_forCNN',
         '6_7_19_wide_5w_forCNN', '6_19_19_wide_5w_forCNN', '6_4_19_wide_5w_forCNN',
         '5_24_19_wide_5w_forCNN','5_18_19_wide_5w_forCNN', '5_20_19_wide_5w_forCNN',
          'E280_64px_w_CNN','E287_64px_w_CNN',
          'E329_64px_w_CNN','E331_64px_w_CNN',
          'E459_64px_w_CNN','E464_64px_w_CNN',
          'E499_64px_w_CNN','E340_64px_w_CNN',
          'E342_64px_w_CNN','E364_64px_w_CNN','E365_64px_w_CNN',
          'E351_64px_w_CNN', 'E352_64px_w_CNN',
        'E440_64px_w_CNN', 'E451_64px_w_CNN', 'E455_64px_w_CNN']
    
    G2 = ['5_22_19_wide_5w_forCNN', '6_18_19_wide_5w_forCNN',
          '5_23_19_wide_5w_forCNN', '5_25_19_wide_5w_forCNN', 
          '6_24_19_wide_5w_forCNN', '6_13_19_wide_5w_forCNN', '6_3_19_wide_5w_forCNN',
          'E319_64px_w_CNN','E320_64px_w_CNN',
          'E321_64px_w_CNN','E322_64px_w_CNN',
          'E324_64px_w_CNN','E328_64px_w_CNN',
          'E332_64px_w_CNN','E345_64px_w_CNN',
          'E346_64px_w_CNN','E347_64px_w_CNN',
          'E353_64px_w_CNN','E355_64px_w_CNN',
          'E458_64px_w_CNN','E461_64px_w_CNN',
          'E463_64px_w_CNN','E338_64px_w_CNN',
          'E339_64px_w_CNN','E363_64px_w_CNN',
          'E366_64px_w_CNN','E368_64px_w_CNN','E369_64px_w_CNN',
          'E350_64px_w_CNN','E439_64px_w_CNN', 
          'E441_64px_w_CNN', 'E442_64px_w_CNN', 
          'E444_64px_w_CNN', 'E452_64px_w_CNN',
          'E456_64px_w_CNN', 'E457_64px_w_CNN',
          'E481_64px_w_CNN', 'E482_64px_w_CNN', 
          'E483_64px_w_CNN', 'E484_64px_w_CNN','E494_64px_w_CNN']
    
    create_dataset(NPY_FOLDER, ML_exp, G1, G2, split_ratio)
          
          
          
    



num_win 5
len_b 681
[0 1 2 3 4]
[0 1 2 4 3]
num_win 5
len_b 478
[0 1 2 3 4]
[4 1 0 3 2]
num_win 5
len_b 702
[0 1 2 3 4]
[4 0 1 2 3]
num_win 5
len_b 574
[0 1 2 3 4]
[2 4 3 0 1]
num_win 5
len_b 467
[0 1 2 3 4]
[4 0 1 2 3]
num_win 5
len_b 451
[0 1 2 3 4]
[4 3 2 0 1]
num_win 5
len_b 453
[0 1 2 3 4]
[1 2 4 3 0]
num_win 6
len_b 122
[0 1 2 3 4 5]
[2 3 4 5 0 1]
num_win 4
len_b 168
[0 1 2 3]
[2 1 0 3]
num_win 6
len_b 218
[0 1 2 3 4 5]
[4 5 1 3 0 2]
num_win 6
len_b 225
[0 1 2 3 4 5]
[4 5 2 1 0 3]
num_win 6
len_b 196
[0 1 2 3 4 5]
[3 5 0 4 2 1]
num_win 6
len_b 192
[0 1 2 3 4 5]
[1 2 4 0 5 3]
num_win 10
len_b 353
[0 1 2 3 4 5 6 7 8 9]
[8 5 6 0 4 7 2 9 3 1]
num_win 5
len_b 175
[0 1 2 3 4]
[4 1 0 2 3]
num_win 6
len_b 149
[0 1 2 3 4 5]
[2 4 5 3 0 1]
num_win 5
len_b 147
[0 1 2 3 4]
[1 2 3 4 0]
num_win 8
len_b 313
[0 1 2 3 4 5 6 7]
[2 0 7 4 5 1 6 3]
num_win 4
len_b 133
[0 1 2 3]
[0 2 1 3]
num_win 6
len_b 258
[0 1 2 3 4 5]
[1 2 3 0 5 4]
num_win 4
len_b 86
[0 1 2 3]
[2 1 3 0]
num_win 5
len_b 215
[0 1 2 3